In [0]:
# Simple CNN for the MNIST Dataset
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
import numpy as np

In [0]:
from scipy.ndimage.interpolation import shift

def shift_image(image, dx, dy):
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image

In [0]:
# load data
(X_raw_train, y_train), (X_raw_test, y_test) = mnist.load_data()
print("Creating Augmented Dataset...")
X_raw_train_augmented = [image for image in X_raw_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((1,0), (-1,0), (0,1), (0,-1), (1,1), (-1,1), (-1,-1), (1,-1)):
     for image, label in zip(X_raw_train, y_train):
             X_raw_train_augmented.append(shift_image(image, dx, dy))
             y_train_augmented.append(label)

X_raw_train = np.array(X_raw_train_augmented, dtype=np.uint8)
y_train_augmented = np.array(y_train_augmented, dtype=np.uint8)

Creating Augmented Dataset...


In [0]:
print("Adding hand labelled data for sudoku grid")
print(X_raw_train.shape, y_train_augmented.shape)
X_raw_train_2 = np.loadtxt("X_train.csv", dtype=np.uint8, delimiter=' ')
y_train_2 = np.loadtxt("y_train.csv", dtype=np.uint8, delimiter=' ')
print(X_raw_train_2.shape, y_train_2.shape)

X_raw_train_2 = np.reshape(X_raw_train_2, (-1, 28, 28))
X_raw_train_2 = 255 - X_raw_train_2   # inverting as hand labelled data set has number is white and back in black

print("Creating Augmented Dataset...")
X_raw_train_augmented_2 = [image for image in X_raw_train_2]
y_train_augmented_2 = [label for label in y_train_2]

for dx, dy in ((1,0), (-1,0), (0,1), (0,-1), (1,1), (-1,1), (-1,-1), (1,-1)):
     for image, label in zip(X_raw_train_2, y_train_2):
             X_raw_train_augmented_2.append(shift_image(image, dx, dy))
             y_train_augmented_2.append(label)

X_raw_train_augmented_2 = np.array(X_raw_train_augmented_2, dtype=np.uint8)
y_train_augmented_2 = np.array(y_train_augmented_2, dtype=np.uint8)

X_raw_train = np.append(X_raw_train, X_raw_train_augmented_2, axis=0)
y_train_augmented = np.append(y_train_augmented, y_train_augmented_2)

Adding hand labelled data for sudoku grid
(540000, 28, 28) (540000,)
(313, 784) (313,)
Creating Augmented Dataset...


In [0]:
# reshape to be [samples][width][height][channels]
X_train = X_raw_train.reshape((X_raw_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_raw_test.reshape((X_raw_test.shape[0], 28, 28, 1)).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

# one hot encode outputs
y_train = np_utils.to_categorical(y_train_augmented)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [0]:
# define a simple CNN model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
	model.add(MaxPooling2D())
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	
  # Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [0]:
# build the model
model_simple = baseline_model()
model_simple.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 4608)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               589952    
_________________________________________________________________
dense_20 (Dense)             (None, 10)                1290      
Total params: 592,074
Trainable params: 592,074
Non-trainable params: 0
_______________________________________________

In [0]:
model_simple.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

# Final evaluation of the model
scores = model_simple.evaluate(X_test, y_test, verbose=1)

print("CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 542817 samples, validate on 10000 samples
Epoch 1/10
542817/542817 [==============================] - 14s 26us/step - loss: 0.0110 - accuracy: 0.9963 - val_loss: 0.0214 - val_accuracy: 0.9941
Epoch 2/10
542817/542817 [==============================] - 14s 25us/step - loss: 0.0082 - accuracy: 0.9972 - val_loss: 0.0193 - val_accuracy: 0.9942
Epoch 3/10
542817/542817 [==============================] - 14s 25us/step - loss: 0.0066 - accuracy: 0.9977 - val_loss: 0.0218 - val_accuracy: 0.9940
Epoch 4/10
542817/542817 [==============================] - 14s 25us/step - loss: 0.0055 - accuracy: 0.9981 - val_loss: 0.0277 - val_accuracy: 0.9929
Epoch 5/10
542817/542817 [==============================] - 14s 25us/step - loss: 0.0050 - accuracy: 0.9982 - val_loss: 0.0237 - val_accuracy: 0.9935
Epoch 6/10
542817/542817 [==============================] - 14s 25us/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 0.0277 - val_accuracy: 0.9930
Epoch 7/10
542817/542817 [=======================

In [0]:
def preprocess_for_CNN(digit: np.ndarray):
  digit = 255 - digit
  digit = digit/255
  digit = digit.reshape((1, 28, 28, 1))
  return digit

In [0]:
X_small_test = np.zeros((X_raw_train_2.shape[0], 28, 28, 1))
for i in range(X_raw_train_2.shape[0]):
  X_small_test[i] = preprocess_for_CNN(X_raw_train_2[i])

# processed_digit = preprocess_for_CNN(X_raw_train_2)
print(X_small_test.shape)

(313, 28, 28, 1)


In [0]:
probabilities_mat = model_simple.predict(X_small_test)
y_pred = np.zeros((probabilities_mat.shape[0], ), dtype=np.uint8)
for i in range(len(probabilities_mat)):
  y_pred[i] = np.argmax(probabilities_mat[i])

In [0]:
from sklearn.metrics import accuracy_score
print(y_pred[0:15])
print(y_train_2[0:15])
print(accuracy_score(y_pred, y_train_2))

[1 8 9 3 4 2 5 3 9 6 4 3 7 1 9]
[1 8 9 3 4 2 5 3 9 6 4 3 7 1 9]
0.9648562300319489


In [0]:
model_simple.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

In [0]:
import tensorflow

In [0]:
tensorflow.__version__

'2.2.0'

In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.3.1'